In [1]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import numpy as np
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
df=pd.read_csv("route_link_data/df_k.csv")
df

,Route_name,Route_link
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,Kozhikode to Bangalore,https://www.redbus.in/bus-tickets/kozhikode-to...
2,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...
3,Ernakulam to Kozhikode,https://www.redbus.in/bus-tickets/ernakulam-to...
4,Bangalore to Kannur,https://www.redbus.in/bus-tickets/bangalore-to...
5,Kozhikode to Mysore,https://www.redbus.in/bus-tickets/kozhikode-to...
6,Kannur to Bangalore,https://www.redbus.in/bus-tickets/kannur-to-ba...
7,Kozhikode to Thiruvananthapuram,https://www.redbus.in/bus-tickets/kozhikode-to...
8,Mysore to Kozhikode,https://www.redbus.in/bus-tickets/mysore-to-ko...
9,Bangalore to Kalpetta (kerala),https://www.redbus.in/bus-tickets/bangalore-to...


KERALA BUS DETAILS

In [25]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Departure_k = []
Arrival_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

#df.iterrows() is used for traversing the Route_link and Route_name simultaneously and is traversed by using 'r' 
for i,r in df.iterrows(): 
    link=r["Route_link"]   # in each index each row is accessed so for example --> Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam
    routes=r["Route_name"]  # in above r["Route_link"] in each index and r['Route_name'] gives the values from the df as Pandas series ... for example if in i = 0 and r['Route_name'] we get Kozhikode to ernakulam 

# Loop through each link
    driver_k.get(link)  #in particular bus link we have to extract...so to load that page we use get()
    time.sleep(2)   #time for waiting before interacting and executing next step dynamically

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  # in href="https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam" the element is loaded 
    for element in elements: 
        element.click()   #the text element has anchor tag so if we click it loads the page where we need to extract the bus details
        time.sleep(2)   #now we have clicked system needs to interact...so if we pause for 2 seconds ...system dynamically gets ready within that time and we can interact further for execution
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")  #the link opens page loads...the view busses button to be clicked 
        clicks.click()  #automatically simulates a mouse click event on the element gathered from XPATH
    except:
        pass 
    time.sleep(2)


    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source    #page_source gets the DOM structure after any dynamic event like click or action performed like scrolling
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  #simulation of keyboard occurs from the actionChain class for user interaction in that particular webdriver instance driver_k.. 
                                                                    #send_keys is a method where Keys.PageDown using keyboard keys helps to scroll long pages easily 
                                                                    #perform - executes the simulation
                                                                    
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source   #DOM Element is stored as string in new_page_source
        
        if new_page_source == old_page_source:
            scrolling = False                    #if new DOM element string is same as old DOM element string



    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")   #each row with each index is iterated using the XPATH, 
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")  
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    

    
    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [26]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Departure_k,
    'End_time': Arrival_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_name': Route_names,
    'Route_link': Route_links
}

df_buses_1 = pd.DataFrame(data)
#convert dataframe to csv
df_buses_1.to_csv('df_bus1.csv',index=False)

In [27]:
df_buses_1

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,MMK Travels,VE A/C Seater / Sleeper (2+1),22:30,07:30,09h 00m,1100,4 Seats available,4.4\n489,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,Kyros Connect,Bharat Benz A/C Semi Sleeper (2+2),23:00,09:00,10h 00m,1099,21 Seats available,4.1\n338,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
2,Suraj Holidays,VE A/C Sleeper (2+1),22:30,07:30,09h 00m,1450,6 Seats available,3.9\n270,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
3,AdFAST LINE BUS,A/C Sleeper (2+1),21:15,06:15,09h 00m,2699,34 Seats available,4.4\n7,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
4,SURAJ TRAVEL HOUSE,VE A/C Seater / Sleeper (2+1),21:00,07:30,10h 30m,949,3 Seats available,4.0\n111,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
...,...,...,...,...,...,...,...,...,...,...
212,Madhavi Travels,A/C Sleeper (2+1),19:00,03:00,08h 00m,999,10 Seats available,3.9,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...
213,Madhavi Travels,Volvo Multi-Axle I-Shift B11R Semi Sleeper (2+2),23:59,04:30,04h 31m,899,26 Seats available,3.9,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...
214,Sonia Travels,A/C Seater/Sleeper (2+1),19:10,02:29,07h 19m,750,36 Seats available,3.9,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...
215,Sonia Travels,A/C Semi Sleeper (2+2),18:30,02:30,08h 00m,750,18 Seats available,3.9,Kottayam to Kozhikode,https://www.redbus.in/bus-tickets/kottayam-to-...


ANDHRA BUS DETAILS

In [4]:
df_a=pd.read_csv("route_link_data/df_a.csv")
df_a

,Route_name,Route_link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...
2,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...
3,Kakinada to Visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
4,Bangalore to Tirupati,https://www.redbus.in/bus-tickets/bangalore-to...
5,Bangalore to Kadapa,https://www.redbus.in/bus-tickets/bangalore-to...
6,Ongole to Hyderabad,https://www.redbus.in/bus-tickets/ongole-to-hy...
7,Kadapa to Bangalore,https://www.redbus.in/bus-tickets/kadapa-to-ba...
8,Chittoor (Andhra Pradesh) to Bangalore,https://www.redbus.in/bus-tickets/chittoor-and...
9,Visakhapatnam to Kakinada,https://www.redbus.in/bus-tickets/visakhapatna...


In [43]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Departure_k = []
Arrival_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

#df.iterrows() is used for traversing the Route_link and Route_name simultaneously and is traversed by using 'r' 
for i,r in df_a.iterrows(): 
    link=r["Route_link"]   # in each index each row is accessed so for example --> Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam
    routes=r["Route_name"]  # in above r["Route_link"] in each index and r['Route_name'] gives the values from the df as Pandas series ... for example if in i = 0 and r['Route_name'] we get Kozhikode to ernakulam 

# Loop through each link
    driver_k.get(link)  #in particular bus link we have to extract...so to load that page we use get()
    time.sleep(2)   #time for waiting before interacting and executing next step dynamically

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  # in href="https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam" the element is loaded 
    for element in elements: 
        element.click()   #the text element has anchor tag so if we click it loads the page where we need to extract the bus details
        time.sleep(2)   #now we have clicked system needs to interact...so if we pause for 2 seconds ...system dynamically gets ready within that time and we can interact further for execution
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")  #the link opens page loads...the view busses button to be clicked 
        clicks.click()  #automatically simulates a mouse click event on the element gathered from XPATH
    except:
        continue
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source    #page_source gets the DOM structure after any dynamic event like click or action performed like scrolling
        
        # Use ActionChains to perform a PAGE_DOWN
        #ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  #simulation of keyboard occurs from the actionChain class for user interaction in that particular webdriver instance driver_k.. 
                                                                    #send_keys is a method where Keys.PageDown using keyboard keys helps to scroll long pages easily 
                                                                    #perform - executes the simulation
                                                                    
        body=driver_k.find_element(By.TAG_NAME,"body")
        body.send_keys(Keys.PAGE_DOWN)

        time.sleep(5)  
        
        new_page_source = driver_k.page_source   #DOM Element is stored as string in new_page_source
        
        if new_page_source == old_page_source:
            scrolling = False                    #if new DOM element string is same as old DOM element string

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")   #each row with each index is iterated using the XPATH, 
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")  
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [47]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Departure_k,
    'End_time': Arrival_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_name': Route_names,
    'Route_link': Route_links
}

df_buses_2 = pd.DataFrame(data)
#convert dataframe to csv
df_buses_2.to_csv("df_bus2.csv",index=False)

In [48]:
df_buses_2

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,TSRTC - 1935,Super Luxury (Non AC Seater 2+2 Push Back),19:00,05:05,10h 05m,550,8 Seats available,1.6\n13,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...
1,TSRTC - 4333,Lahari Non A/C Sleeper Cum Seater,21:45,05:25,07h 40m,550,4 Seats available,3.4\n177,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...
2,TSRTC - 4335,Lahari Non A/C Sleeper Cum Seater,22:15,05:55,07h 40m,870,7 Seats available,3.5\n154,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...
3,V Kaveri Travels,Scania AC Multi Axle Sleeper (2+1),21:30,05:45,08h 15m,1539,13 Seats available,4.2\n250,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...
4,V Kaveri Travels,A/C Sleeper (2+1),21:30,06:15,08h 45m,1490,16 Seats available,4.0\n70,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...
...,...,...,...,...,...,...,...,...,...,...
467,BRS Travels,A/C Seater / Sleeper (2+1),23:00,06:30,07h 30m,509,15 Seats available,3.5\n351,Kurnool to Bangalore,https://www.redbus.in/bus-tickets/kurnool-to-b...
468,Yolo Bus,Bharat Benz A/C Sleeper (2+1),23:46,07:54,08h 08m,910,16 Seats available,3.8\n71,Kurnool to Bangalore,https://www.redbus.in/bus-tickets/kurnool-to-b...
469,YAS TOURS AND TRAVELS,Volvo A/C Semi Sleeper (2+2),19:45,01:15,05h 30m,700,13 Seats available,3.9\n63,Kurnool to Bangalore,https://www.redbus.in/bus-tickets/kurnool-to-b...
470,BRS Travels,A/C Sleeper (2+1),23:15,06:15,07h 00m,648,19 Seats available,3.9\n264,Kurnool to Bangalore,https://www.redbus.in/bus-tickets/kurnool-to-b...


TELENGANA BUS DETAILS

In [5]:
df_t=pd.read_csv("route_link_data/df_t.csv")
df_t

,Route_name,Route_link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
2,Hyderabad to Khammam,https://www.redbus.in/bus-tickets/hyderabad-to...
3,Hyderabad to Srisailam,https://www.redbus.in/bus-tickets/hyderabad-to...
4,Karimnagar to Hyderabad,https://www.redbus.in/bus-tickets/karimnagar-t...
5,Hyderabad to Karimnagar,https://www.redbus.in/bus-tickets/hyderabad-to...
6,Hyderabad to Mancherial,https://www.redbus.in/bus-tickets/hyderabad-to...
7,Hyderabad to Nirmal,https://www.redbus.in/bus-tickets/hyderabad-to...
8,Hyderabad to Adilabad,https://www.redbus.in/bus-tickets/hyderabad-to...
9,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...


In [50]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Departure_k = []
Arrival_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

#df.iterrows() is used for traversing the Route_link and Route_name simultaneously and is traversed by using 'r' 
for i,r in df_t.iterrows(): 
    link=r["Route_link"]   # in each index each row is accessed so for example --> Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam
    routes=r["Route_name"]  # in above r["Route_link"] in each index and r['Route_name'] gives the values from the df as Pandas series ... for example if in i = 0 and r['Route_name'] we get Kozhikode to ernakulam 

# Loop through each link
    driver_k.get(link)  #in particular bus link we have to extract...so to load that page we use get()
    time.sleep(2)   #time for waiting before interacting and executing next step dynamically

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  # in href="https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam" the element is loaded 
    for element in elements: 
        element.click()   #the text element has anchor tag so if we click it loads the page where we need to extract the bus details
        time.sleep(2)   #now we have clicked system needs to interact...so if we pause for 2 seconds ...system dynamically gets ready within that time and we can interact further for execution
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")  #the link opens page loads...the view busses button to be clicked 
        clicks.click()  #automatically simulates a mouse click event on the element gathered from XPATH
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source    #page_source gets the DOM structure after any dynamic event like click or action performed like scrolling
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  #simulation of keyboard occurs from the actionChain class for user interaction in that particular webdriver instance driver_k.. 
                                                                    #send_keys is a method where Keys.PageDown using keyboard keys helps to scroll long pages easily 
                                                                    #perform - executes the simulation
                                                                    
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source   #DOM Element is stored as string in new_page_source
        
        if new_page_source == old_page_source:
            scrolling = False                    #if new DOM element string is same as old DOM element string

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")   #each row with each index is iterated using the XPATH, 
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")  
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [51]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Departure_k,
    'End_time': Arrival_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_name': Route_names,
    'Route_link': Route_links
}

df_buses_3 = pd.DataFrame(data)
#convert dataframe to csv
df_buses_3.to_csv("df_bus3.csv",index=False)

In [52]:
df_buses_3

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,TSRTC - 1952,LAHARI A/C SLEEPER CUM SEATER,18:10,21:45,03h 35m,389,26 Seats available,3.2\n42,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
1,TSRTC - 4172,RAJADHANI AC (CONVERTED METRO LUXURY),18:30,22:30,04h 00m,389,33 Seats available,3.7\n44,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
2,TSRTC - 64022,RAJADHANI AC (CONVERTED METRO LUXURY),19:30,00:15,04h 45m,389,33 Seats available,2.9\n27,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
3,TSRTC - 8677,Super Luxury (Non AC Seater 2+2 Push Back),19:30,23:40,04h 10m,320,31 Seats available,3.9\n20,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
4,TSRTC - 8625,RAJDHANI (A.C. Semi Sleeper),21:00,01:00,04h 00m,389,29 Seats available,3.5\n52,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
...,...,...,...,...,...,...,...,...,...,...
272,Morning Star Travels,A/C Sleeper (2+1),18:30,01:30,07h 00m,1290,12 Seats available,4.3,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...
273,PMR Express,A/C Sleeper (2+1),19:00,02:00,07h 00m,999,18 Seats available,4.0,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...
274,Sri Vengamamba Bus Transport (SVBT),A/C Seater / Sleeper (2+1),21:40,05:30,07h 50m,1041,4 Seats available,4.6\n39,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...
275,Kaveri Travels,Non A/C Seater / Sleeper (2+1),21:35,05:15,07h 40m,890,2 Seats available,4.6\n25,Hyderabad to Addanki,https://www.redbus.in/bus-tickets/hyderabad-to...


ASSAM BUS DETAILS

In [6]:
df_as=pd.read_csv("route_link_data/df_as.csv")
df_as

,Route_name,Route_link
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
1,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...
2,Nagaon (Assam) to Guwahati,https://www.redbus.in/bus-tickets/nagaon-to-gu...
3,Guwahati to Nagaon (Assam),https://www.redbus.in/bus-tickets/guwahati-to-...
4,Goalpara to Guwahati,https://www.redbus.in/bus-tickets/goalpara-to-...
5,Jorhat to North Lakhimpur,https://www.redbus.in/bus-tickets/jorhat-to-no...
6,Dhubri to Guwahati,https://www.redbus.in/bus-tickets/dhubri-to-gu...
7,Guwahati to Dhubri,https://www.redbus.in/bus-tickets/guwahati-to-...
8,North Lakhimpur to Sibsagar,https://www.redbus.in/bus-tickets/north-lakhim...
9,North Lakhimpur to Jorhat,https://www.redbus.in/bus-tickets/north-lakhim...


In [29]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Departure_k = []
Arrival_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

#df.iterrows() is used for traversing the Route_link and Route_name simultaneously and is traversed by using 'r' 
for i,r in df_as.iterrows(): 
    link=r["Route_link"]   # in each index each row is accessed so for example --> Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam
    routes=r["Route_name"]  # in above r["Route_link"] in each index and r['Route_name'] gives the values from the df as Pandas series ... for example if in i = 0 and r['Route_name'] we get Kozhikode to ernakulam 

# Loop through each link
    driver_k.get(link)  #in particular bus link we have to extract...so to load that page we use get()
    time.sleep(2)   #time for waiting before interacting and executing next step dynamically

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  # in href="https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam" the element is loaded 
    for element in elements: 
        element.click()   #the text element has anchor tag so if we click it loads the page where we need to extract the bus details
        time.sleep(2)   #now we have clicked system needs to interact...so if we pause for 2 seconds ...system dynamically gets ready within that time and we can interact further for execution
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")  #the link opens page loads...the view busses button to be clicked 
        clicks.click()  #automatically simulates a mouse click event on the element gathered from XPATH
    except:
        pass 
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source    #page_source gets the DOM structure after any dynamic event like click or action performed like scrolling
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  #simulation of keyboard occurs from the actionChain class for user interaction in that particular webdriver instance driver_k.. 
                                                                    #send_keys is a method where Keys.PageDown using keyboard keys helps to scroll long pages easily 
                                                                    #perform - executes the simulation
                                                                    
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source   #DOM Element is stored as string in new_page_source
        
        if new_page_source == old_page_source:
            scrolling = False                    #if new DOM element string is same as old DOM element string

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")   #each row with each index is iterated using the XPATH, 
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")  
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [31]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Departure_k,
    'End_time': Arrival_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_name': Route_names,
    'Route_link': Route_links
}

df_buses_4 = pd.DataFrame(data)
#convert dataframe to csv
df_buses_4.to_csv("df_bus4.csv",index=False)

In [32]:
df_buses_4

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),22:15,03:45,05h 30m,450,17 Seats available,3.3\n27,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
1,BLUE HILL TRAVELS INDIA LIMITED,NON A/C Seater Semi Sleeper (2+1),21:30,03:30,06h 00m,400,15 Seats available,2.9\n96,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
2,Himalayan Travels,NON A/C Seater (2+2),23:40,04:00,04h 20m,450,26 Seats available,2.2\n14,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
3,Yatra Travels,NON A/C Seater (2+1),18:00,23:30,05h 30m,350,17 Seats available,3.7\n24,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...
4,Yatra Travels,NON AC Seater/ Sleeper (2+1),20:00,00:00,04h 00m,350,24 Seats available,4.0\n30,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...
...,...,...,...,...,...,...,...,...,...,...
85,Puja Transport,Non AC Seater (2+2),20:00,06:00,10h 00m,550,8 Seats available,3.2\n20,Tezpur to Tinsukia,https://www.redbus.in/bus-tickets/tezpur-to-ti...
86,Puja Transport,NON AC Seater / Sleeper 2+1,20:00,05:30,09h 30m,700,1 Seat available,2.2\n18,Tezpur to Tinsukia,https://www.redbus.in/bus-tickets/tezpur-to-ti...
87,Neha Travels,NON A/C Seater (2+2),22:20,08:00,09h 40m,650,1 Seat available,1.4\n4,Tezpur to Tinsukia,https://www.redbus.in/bus-tickets/tezpur-to-ti...
88,Baikuntha Transport Service,Bharat Benz A/C Sleeper (2+1),19:30,01:00,05h 30m,500,34 Seats available,3.9\n7,Dibrugarh to Bokakhat,https://www.redbus.in/bus-tickets/dibrugarh-to...


GOA BUS DETAILS

In [7]:
df_g=pd.read_csv("route_link_data/df_g.csv")
df_g

,Route_name,Route_link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
2,Hyderabad to Khammam,https://www.redbus.in/bus-tickets/hyderabad-to...
3,Hyderabad to Srisailam,https://www.redbus.in/bus-tickets/hyderabad-to...
4,Karimnagar to Hyderabad,https://www.redbus.in/bus-tickets/karimnagar-t...
5,Hyderabad to Karimnagar,https://www.redbus.in/bus-tickets/hyderabad-to...
6,Hyderabad to Mancherial,https://www.redbus.in/bus-tickets/hyderabad-to...
7,Hyderabad to Nirmal,https://www.redbus.in/bus-tickets/hyderabad-to...
8,Hyderabad to Adilabad,https://www.redbus.in/bus-tickets/hyderabad-to...
9,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...


In [34]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Departure_k = []
Arrival_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

#df.iterrows() is used for traversing the Route_link and Route_name simultaneously and is traversed by using 'r' 
for i,r in df_g.iterrows(): 
    link=r["Route_link"]   # in each index each row is accessed so for example --> Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam
    routes=r["Route_name"]  # in above r["Route_link"] in each index and r['Route_name'] gives the values from the df as Pandas series ... for example if in i = 0 and r['Route_name'] we get Kozhikode to ernakulam 

# Loop through each link
    driver_k.get(link)  #in particular bus link we have to extract...so to load that page we use get()
    time.sleep(2)   #time for waiting before interacting and executing next step dynamically

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  # in href="https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam" the element is loaded 
    for element in elements: 
        element.click()   #the text element has anchor tag so if we click it loads the page where we need to extract the bus details
        time.sleep(2)   #now we have clicked system needs to interact...so if we pause for 2 seconds ...system dynamically gets ready within that time and we can interact further for execution
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")  #the link opens page loads...the view busses button to be clicked 
        clicks.click()  #automatically simulates a mouse click event on the element gathered from XPATH
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source    #page_source gets the DOM structure after any dynamic event like click or action performed like scrolling
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  #simulation of keyboard occurs from the actionChain class for user interaction in that particular webdriver instance driver_k.. 
                                                                    #send_keys is a method where Keys.PageDown using keyboard keys helps to scroll long pages easily 
                                                                    #perform - executes the simulation
                                                                    
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source   #DOM Element is stored as string in new_page_source
        
        if new_page_source == old_page_source:
            scrolling = False                    #if new DOM element string is same as old DOM element string

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")   #each row with each index is iterated using the XPATH, 
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")  
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [35]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Departure_k,
    'End_time': Arrival_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_name': Route_names,
    'Route_link': Route_links
}

df_buses_5 = pd.DataFrame(data)
#convert dataframe to csv
df_buses_5.to_csv("df_bus5.csv",index=False)

In [36]:
df_buses_5

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,TSRTC - 8675,RAJDHANI (A.C. Semi Sleeper),17:35,21:30,03h 55m,389,25 Seats available,3.8\n57,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
1,TSRTC - 64089,RAJDHANI (A.C. Semi Sleeper),18:00,22:00,04h 00m,389,33 Seats available,3.3\n34,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
2,TSRTC - 1952,LAHARI A/C SLEEPER CUM SEATER,18:10,21:45,03h 35m,389,23 Seats available,3.3\n41,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
3,TSRTC - 4172,RAJADHANI AC (CONVERTED METRO LUXURY),18:30,22:30,04h 00m,389,35 Seats available,3.7\n44,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
4,TSRTC - 8677,Super Luxury (Non AC Seater 2+2 Push Back),19:30,23:40,04h 10m,320,31 Seats available,3.9\n20,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
...,...,...,...,...,...,...,...,...,...,...
134,Vahini Travels,NON A/C Push Back (2+2),21:10,03:30,06h 20m,670,17 Seats available,3.7,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...
135,Vahini Travels,Non A/C Seater / Sleeper (2+1),21:10,03:30,06h 20m,780,20 Seats available,3.7,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...
136,LVP Travels,A/C Seater / Sleeper (2+1),21:15,02:50,05h 35m,780,21 Seats available,4.1,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...
137,Sri KVR Travels,Non A/C Seater / Sleeper (2+1),21:30,02:45,05h 15m,780,8 Seats available,4.0,Hyderabad to Kodad,https://www.redbus.in/bus-tickets/hyderabad-to...


HARYANA BUS DETAILS

In [8]:
df_h=pd.read_csv("route_link_data/df_h.csv")
df_h

,Route_name,Route_link
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla
1,Shimla to Delhi,https://www.redbus.in/bus-tickets/shimla-to-delhi
2,Manali to Chandigarh,https://www.redbus.in/bus-tickets/manali-to-ch...
3,Chandigarh to Manali,https://www.redbus.in/bus-tickets/chandigarh-t...
4,Delhi to Manali,https://www.redbus.in/bus-tickets/delhi-to-manali
5,Hamirpur (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/hamirpur-him...
6,Delhi to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-ham...
7,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...
8,Manali to Delhi,https://www.redbus.in/bus-tickets/manali-to-delhi
9,Hamirpur (Himachal Pradesh) to Delhi,https://www.redbus.in/bus-tickets/hamirpur-him...


In [39]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Departure_k = []
Arrival_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

#df.iterrows() is used for traversing the Route_link and Route_name simultaneously and is traversed by using 'r' 
for i,r in df_h.iterrows(): 
    link=r["Route_link"]   # in each index each row is accessed so for example --> Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam
    routes=r["Route_name"]  # in above r["Route_link"] in each index and r['Route_name'] gives the values from the df as Pandas series ... for example if in i = 0 and r['Route_name'] we get Kozhikode to ernakulam 

# Loop through each link
    driver_k.get(link)  #in particular bus link we have to extract...so to load that page we use get()
    time.sleep(2)   #time for waiting before interacting and executing next step dynamically

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  # in href="https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam" the element is loaded 
    for element in elements: 
        element.click()   #the text element has anchor tag so if we click it loads the page where we need to extract the bus details
        time.sleep(2)   #now we have clicked system needs to interact...so if we pause for 2 seconds ...system dynamically gets ready within that time and we can interact further for execution
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")  #the link opens page loads...the view busses button to be clicked 
        clicks.click()  #automatically simulates a mouse click event on the element gathered from XPATH
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source    #page_source gets the DOM structure after any dynamic event like click or action performed like scrolling
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  #simulation of keyboard occurs from the actionChain class for user interaction in that particular webdriver instance driver_k.. 
                                                                    #send_keys is a method where Keys.PageDown using keyboard keys helps to scroll long pages easily 
                                                                    #perform - executes the simulation
                                                                    
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source   #DOM Element is stored as string in new_page_source
        
        if new_page_source == old_page_source:
            scrolling = False                    #if new DOM element string is same as old DOM element string

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")   #each row with each index is iterated using the XPATH, 
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")  
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [40]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Departure_k,
    'End_time': Arrival_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_name': Route_names,
    'Route_link': Route_links
}

df_buses_6 = pd.DataFrame(data)
#convert dataframe to csv
df_buses_6.to_csv("df_bus6.csv",index=False)

In [41]:
df_buses_6

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,HRTC - 93,Himsuta AC Seater Volvo/Scania 2+2,19:35,04:45,09h 10m,892,7 Seats available,3.4\n37,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla
1,HRTC - 301,Ordinary 3+2 Non AC Seater,19:43,05:30,09h 47m,507,27 Seats available,4.5\n4,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla
2,HRTC - 556,Ordinary 3+2 Non AC Seater,20:10,05:45,09h 35m,507,21 Seats available,4.5\n10,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla
3,HRTC - 591,A/C Executive (2+3),21:00,07:20,10h 20m,622,40 Seats available,3.8\n44,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla
4,HRTC - 8,Himsuta AC Seater Volvo/Scania 2+2,21:02,06:30,09h 28m,892,10 Seats available,4.6\n89,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla
...,...,...,...,...,...,...,...,...,...,...
322,HRTC - 750,A/C Executive (2+3),20:30,06:00,09h 30m,600,44 Seats available,2.8\n21,Ghumarwin to Delhi,https://www.redbus.in/bus-tickets/ghumarwin-to...
323,HRTC - 1107,Ordinary 3+2 Non AC Seater,20:30,04:15,07h 45m,463,36 Seats available,2.2\n4,Ghumarwin to Delhi,https://www.redbus.in/bus-tickets/ghumarwin-to...
324,HRTC - 78,Himsuta AC Seater Volvo/Scania 2+2,21:15,05:00,07h 45m,912,20 Seats available,4.4\n55,Ghumarwin to Delhi,https://www.redbus.in/bus-tickets/ghumarwin-to...
325,HRTC - 15,Himsuta AC Seater Volvo/Scania 2+2,21:25,04:30,07h 05m,856,20 Seats available,3.5\n10,Ghumarwin to Delhi,https://www.redbus.in/bus-tickets/ghumarwin-to...


PUNJAB BUS DETAILS

In [9]:
df_pb=pd.read_csv("route_link_data/df_pb.csv")
df_pb

,Route_name,Route_link
0,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...
1,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...
2,Ludhiana to Delhi,https://www.redbus.in/bus-tickets/ludhiana-to-...
3,Delhi to Ludhiana,https://www.redbus.in/bus-tickets/delhi-to-lud...
4,Phagwara to Delhi,https://www.redbus.in/bus-tickets/phagwara-to-...
5,Jalandhar to Delhi,https://www.redbus.in/bus-tickets/jalandhar-to...
6,Delhi to Jalandhar,https://www.redbus.in/bus-tickets/delhi-to-jal...
7,Patiala to Delhi Airport,https://www.redbus.in/bus-tickets/patiala-to-d...
8,Jalandhar to Delhi Airport,https://www.redbus.in/bus-tickets/jalandhar-to...
9,Ludhiana to Delhi Airport,https://www.redbus.in/bus-tickets/ludhiana-to-...


In [46]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Departure_k = []
Arrival_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

#df.iterrows() is used for traversing the Route_link and Route_name simultaneously and is traversed by using 'r' 
for i,r in df_pb.iterrows(): 
    link=r["Route_link"]   # in each index each row is accessed so for example --> Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam
    routes=r["Route_name"]  # in above r["Route_link"] in each index and r['Route_name'] gives the values from the df as Pandas series ... for example if in i = 0 and r['Route_name'] we get Kozhikode to ernakulam 

# Loop through each link
    driver_k.get(link)  #in particular bus link we have to extract...so to load that page we use get()
    time.sleep(2)   #time for waiting before interacting and executing next step dynamically

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  # in href="https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam" the element is loaded 
    for element in elements: 
        element.click()   #the text element has anchor tag so if we click it loads the page where we need to extract the bus details
        time.sleep(2)   #now we have clicked system needs to interact...so if we pause for 2 seconds ...system dynamically gets ready within that time and we can interact further for execution
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")  #the link opens page loads...the view busses button to be clicked 
        clicks.click()  #automatically simulates a mouse click event on the element gathered from XPATH
    except:
        pass 
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source    #page_source gets the DOM structure after any dynamic event like click or action performed like scrolling
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  #simulation of keyboard occurs from the actionChain class for user interaction in that particular webdriver instance driver_k.. 
                                                                    #send_keys is a method where Keys.PageDown using keyboard keys helps to scroll long pages easily 
                                                                    #perform - executes the simulation
                                                                    
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source   #DOM Element is stored as string in new_page_source
        
        if new_page_source == old_page_source:
            scrolling = False                    #if new DOM element string is same as old DOM element string

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")   #each row with each index is iterated using the XPATH, 
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")  
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [47]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Departure_k,
    'End_time': Arrival_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_name': Route_names,
    'Route_link': Route_links
}

df_buses_7 = pd.DataFrame(data)
#convert dataframe to csv
df_buses_7.to_csv("df_bus7.csv",index=False)

In [48]:
df_buses_7

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,PEPSU (Punjab) - 67104,AC Seater Hvac 2+2,22:30,03:31,05h 01m,440,39 Seats available,3.1\n163,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...
1,PEPSU (Punjab) - 67103,AC Seater Hvac 2+2,22:31,04:00,05h 29m,440,12 Seats available,3.2\n237,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...
2,Yolo Bus,VE A/C Seater / Sleeper (2+1),22:27,05:00,06h 33m,400,5 Seats available,4.6\n457,Ludhiana to Delhi,https://www.redbus.in/bus-tickets/ludhiana-to-...
3,Zingbus Plus,Bharat Benz A/C Seater /Sleeper (2+1),23:50,06:35,06h 45m,419,10 Seats available,4.7\n104,Ludhiana to Delhi,https://www.redbus.in/bus-tickets/ludhiana-to-...
4,IntrCity SmartBus,AC Sleeper (2+1),22:50,05:45,06h 55m,592,7 Seats available,4.6\n160,Ludhiana to Delhi,https://www.redbus.in/bus-tickets/ludhiana-to-...
...,...,...,...,...,...,...,...,...,...,...
122,Zimindara Travels,AC Sleeper (2+1),20:30,01:50,05h 20m,810,17 Seats available,4.0,Chandigarh to Bathinda,https://www.redbus.in/bus-tickets/chandigarh-t...
123,Rajpreet Travels and cargo,A/C Sleeper (2+1),20:40,01:30,04h 50m,905,12 Seats available,3.5,Chandigarh to Bathinda,https://www.redbus.in/bus-tickets/chandigarh-t...
124,Rajpreet Travels and cargo,VE A/C Sleeper (2+1),21:20,02:00,04h 40m,905,9 Seats available,3.5,Chandigarh to Bathinda,https://www.redbus.in/bus-tickets/chandigarh-t...
125,Shree Balaji Express,A/C Seater/Sleeper (2+1),20:00,01:00,05h 00m,599,35 Seats available,2.6,Chandigarh to Bathinda,https://www.redbus.in/bus-tickets/chandigarh-t...


RAJASTHAN BUS DETAILS

In [10]:
df_r=pd.read_csv("route_link_data/df_r.csv")
df_r

,Route_name,Route_link
0,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
1,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...
2,Beawar (Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/beawer-to-ja...
3,Sikar to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/sikar-to-jaipur
4,Jaipur (Rajasthan) to Jodhpur,https://www.redbus.in/bus-tickets/jaipur-to-jo...
5,Aligarh (uttar pradesh) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/aligarh-utta...
6,Jaipur (Rajasthan) to Aligarh (uttar pradesh),https://www.redbus.in/bus-tickets/jaipur-to-al...
7,Jodhpur to Beawar (Rajasthan),https://www.redbus.in/bus-tickets/jodhpur-to-b...
8,Jaipur (Rajasthan) to Pilani,https://www.redbus.in/bus-tickets/jaipur-to-pi...
9,Kishangarh to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/kishangarh-t...


In [50]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Departure_k = []
Arrival_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

#df.iterrows() is used for traversing the Route_link and Route_name simultaneously and is traversed by using 'r' 
for i,r in df_r.iterrows(): 
    link=r["Route_link"]   # in each index each row is accessed so for example --> Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam
    routes=r["Route_name"]  # in above r["Route_link"] in each index and r['Route_name'] gives the values from the df as Pandas series ... for example if in i = 0 and r['Route_name'] we get Kozhikode to ernakulam 

# Loop through each link
    driver_k.get(link)  #in particular bus link we have to extract...so to load that page we use get()
    time.sleep(2)   #time for waiting before interacting and executing next step dynamically

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  # in href="https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam" the element is loaded 
    for element in elements: 
        element.click()   #the text element has anchor tag so if we click it loads the page where we need to extract the bus details
        time.sleep(2)   #now we have clicked system needs to interact...so if we pause for 2 seconds ...system dynamically gets ready within that time and we can interact further for execution
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")  #the link opens page loads...the view busses button to be clicked 
        clicks.click()  #automatically simulates a mouse click event on the element gathered from XPATH
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source    #page_source gets the DOM structure after any dynamic event like click or action performed like scrolling
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  #simulation of keyboard occurs from the actionChain class for user interaction in that particular webdriver instance driver_k.. 
                                                                    #send_keys is a method where Keys.PageDown using keyboard keys helps to scroll long pages easily 
                                                                    #perform - executes the simulation
                                                                    
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source   #DOM Element is stored as string in new_page_source
        
        if new_page_source == old_page_source:
            scrolling = False                    #if new DOM element string is same as old DOM element string

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")   #each row with each index is iterated using the XPATH, 
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")  
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [51]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Departure_k,
    'End_time': Arrival_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_name': Route_names,
    'Route_link': Route_links
}

df_buses_8 = pd.DataFrame(data)
#convert dataframe to csv
df_buses_8.to_csv("df_bus8.csv",index=False)

In [52]:
df_buses_8

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,RSRTC - 189473,NON AC Seater / Sleeper 2+1,21:00,01:30,04h 30m,302,37 Seats available,3.9,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...
1,RSRTC - 192299,Super Luxury Volvo AC Seater Pushback 2+2,23:00,02:45,03h 45m,458,24 Seats available,4.2\n8,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...
2,RSRTC - 189583,NON AC Seater / Sleeper 2+1,23:59,03:45,03h 46m,302,26 Seats available,4.6\n6,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...
3,Jain travels regd,NON AC Seater / Sleeper 2+1,19:20,23:50,04h 30m,330,19 Seats available,3.1,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...
4,FlixBus,A/C Sleeper (2+1),19:45,23:50,04h 05m,186,27 Seats available,5.0,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...
...,...,...,...,...,...,...,...,...,...,...
105,Mansoori tour and travels,A/C Seater / Sleeper (2+1),21:00,01:45,04h 45m,630,40 Seats available,1.7,Jaipur (Rajasthan) to Mathura,https://www.redbus.in/bus-tickets/jaipur-to-ma...
106,janta shatabdi travels,Non A/C Seater / Sleeper (2+1),21:15,01:59,04h 44m,463,40 Seats available,1.4,Jaipur (Rajasthan) to Mathura,https://www.redbus.in/bus-tickets/jaipur-to-ma...
107,SHAGUN CARGO & TRAVELS,A/C Sleeper (2+1),21:15,02:45,05h 30m,570,22 Seats available,2.0\n6,Jaipur (Rajasthan) to Mathura,https://www.redbus.in/bus-tickets/jaipur-to-ma...
108,janta shatabdi travels,Non A/C Seater / Sleeper (2+1),21:15,01:59,04h 44m,463,47 Seats available,1.4,Jaipur (Rajasthan) to Mathura,https://www.redbus.in/bus-tickets/jaipur-to-ma...


SOUTH BENGAL BUS DETAILS

In [11]:
df_sb=pd.read_csv("route_link_data/df_sb.csv")
df_sb


,Route_name,Route_link
0,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...
1,Kolkata to Burdwan,https://www.redbus.in/bus-tickets/kolkata-to-b...
2,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...
3,Kolkata to Haldia,https://www.redbus.in/bus-tickets/kolkata-to-h...
4,Haldia to Kolkata,https://www.redbus.in/bus-tickets/haldia-to-ko...
5,Kolkata to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-d...
6,Kolkata to Arambagh (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-a...
7,Digha to Kolkata,https://www.redbus.in/bus-tickets/digha-to-kol...
8,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...
9,Kolkata to Bankura,https://www.redbus.in/bus-tickets/kolkata-to-b...


In [77]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Departure_k = []
Arrival_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

#df.iterrows() is used for traversing the Route_link and Route_name simultaneously and is traversed by using 'r' 
for i,r in df_sb.iterrows(): 
    link=r["Route_link"]   # in each index each row is accessed so for example --> Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam
    routes=r["Route_name"]  # in above r["Route_link"] in each index and r['Route_name'] gives the values from the df as Pandas series ... for example if in i = 0 and r['Route_name'] we get Kozhikode to ernakulam 

# Loop through each link
    driver_k.get(link)  #in particular bus link we have to extract...so to load that page we use get()
    time.sleep(2)   #time for waiting before interacting and executing next step dynamically

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  # in href="https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam" the element is loaded 
    for element in elements: 
        element.click()   #the text element has anchor tag so if we click it loads the page where we need to extract the bus details
        time.sleep(2)   #now we have clicked system needs to interact...so if we pause for 2 seconds ...system dynamically gets ready within that time and we can interact further for execution
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")  #the link opens page loads...the view busses button to be clicked 
        clicks.click()  #automatically simulates a mouse click event on the element gathered from XPATH
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source    #page_source gets the DOM structure after any dynamic event like click or action performed like scrolling
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  #simulation of keyboard occurs from the actionChain class for user interaction in that particular webdriver instance driver_k.. 
                                                                    #send_keys is a method where Keys.PageDown using keyboard keys helps to scroll long pages easily 
                                                                    #perform - executes the simulation
                                                                    
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source   #DOM Element is stored as string in new_page_source
        
        if new_page_source == old_page_source:
            scrolling = False                    #if new DOM element string is same as old DOM element string

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")   #each row with each index is iterated using the XPATH, 
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")  
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [78]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Departure_k,
    'End_time': Arrival_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_name': Route_names,
    'Route_link': Route_links
}

df_buses_9 = pd.DataFrame(data)
#convert dataframe to csv
df_buses_9.to_csv("df_bus9.csv",index=False)

In [79]:
df_buses_9

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,SBSTC-BURDWAN - KARUNAMOYEE - 17:30 (BWN DEPOT...,Non AC Seater (2+3),17:30,20:00,02h 30m,100,37 Seats available,3.8\n147,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...
1,SBSTC-BURDWAN - KARUNAMOYEE - 18:30 (BWN DEPOT...,Non AC Seater (2+3),18:30,21:00,02h 30m,100,44 Seats available,4.1\n78,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...
2,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo B11R Multi Axle Seater (2+2),17:00,20:00,03h 00m,429,32 Seats available,3.7\n84,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...
3,Express Line (Karunamoyee),Volvo 9600 A/C Seater (2+2),17:40,20:25,02h 45m,429,28 Seats available,4.6\n42,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...
4,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo Multi-Axle A/C Semi Sleeper (2+2),17:45,20:45,03h 00m,429,13 Seats available,3.8\n28,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...
5,Shyamoli Paribahan Pvt Ltd,Volvo Multi-Axle A/C Semi Sleeper (2+2),17:55,20:45,02h 50m,429,12 Seats available,3.8\n32,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...
6,Greenline,Volvo A/C Seater (2+2),18:15,21:05,02h 50m,429,24 Seats available,4.1,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...
7,Greenline (Karunamoyee),Volvo A/C Seater (2+2),18:30,21:20,02h 50m,429,28 Seats available,3.2\n12,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...
8,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo A/C Seater (2+2),19:10,21:50,02h 40m,429,29 Seats available,3.7\n35,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...
9,Express Line,Volvo 9600 Multi Axle Semi-Sleeper (2+2),19:40,22:50,03h 10m,429,38 Seats available,4.2\n16,Burdwan to Kolkata,https://www.redbus.in/bus-tickets/burdwan-to-k...


UTTAR PRADESH BUS DETAILS

In [12]:
df_up=pd.read_csv("route_link_data/df_up.csv")
df_up

,Route_name,Route_link
0,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...
1,Aligarh (uttar pradesh) to Delhi,https://www.redbus.in/bus-tickets/aligarh-utta...
2,Delhi to Bareilly,https://www.redbus.in/bus-tickets/delhi-to-bar...
3,Delhi to Aligarh (uttar pradesh),https://www.redbus.in/bus-tickets/delhi-to-ali...
4,Farrukhabad (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/farrukhabad-...
5,Badaun to Delhi,https://www.redbus.in/bus-tickets/badaun-to-delhi
6,Lucknow to Allahabad,https://www.redbus.in/bus-tickets/lucknow-to-a...
7,Lucknow to Varanasi,https://www.redbus.in/bus-tickets/lucknow-to-v...
8,Delhi to Badaun,https://www.redbus.in/bus-tickets/delhi-to-badaun
9,Agra to Bareilly,https://www.redbus.in/bus-tickets/agra-to-bare...


In [71]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Departure_k = []
Arrival_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

#df.iterrows() is used for traversing the Route_link and Route_name simultaneously and is traversed by using 'r' 
for i,r in df_up.iterrows(): 
    link=r["Route_link"]   # in each index each row is accessed so for example --> Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam
    routes=r["Route_name"]  # in above r["Route_link"] in each index and r['Route_name'] gives the values from the df as Pandas series ... for example if in i = 0 and r['Route_name'] we get Kozhikode to ernakulam 

# Loop through each link
    driver_k.get(link)  #in particular bus link we have to extract...so to load that page we use get()
    time.sleep(2)   #time for waiting before interacting and executing next step dynamically

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  # in href="https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam" the element is loaded 
    for element in elements: 
        element.click()   #the text element has anchor tag so if we click it loads the page where we need to extract the bus details
        time.sleep(2)   #now we have clicked system needs to interact...so if we pause for 2 seconds ...system dynamically gets ready within that time and we can interact further for execution
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")  #the link opens page loads...the view busses button to be clicked 
        clicks.click()  #automatically simulates a mouse click event on the element gathered from XPATH
    except:
        continue  
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source    #page_source gets the DOM structure after any dynamic event like click or action performed like scrolling
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  #simulation of keyboard occurs from the actionChain class for user interaction in that particular webdriver instance driver_k.. 
                                                                    #send_keys is a method where Keys.PageDown using keyboard keys helps to scroll long pages easily 
                                                                    #perform - executes the simulation
                                                                    
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source   #DOM Element is stored as string in new_page_source
        
        if new_page_source == old_page_source:
            scrolling = False                    #if new DOM element string is same as old DOM element string

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")   #each row with each index is iterated using the XPATH, 
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")  
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [72]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Departure_k,
    'End_time': Arrival_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_name': Route_names,
    'Route_link': Route_links
}

df_buses_10 = pd.DataFrame(data)
#convert dataframe to csv
df_buses_10.to_csv("df_bus10.csv",index=False)

In [73]:
df_buses_10

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,UPSRTC - RKD0130,Janrath AC Seater 2+3,17:31,22:31,05h 00m,520,47 Seats available,3.4\n12,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...
1,UPSRTC - BLY0111,Janrath AC Seater 2+2,19:01,23:45,04h 44m,598,36 Seats available,2.1\n5,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...
2,UPSRTC - HDD0063,Ordinary Non AC Seater 2+3,19:39,01:03,05h 24m,415,52 Seats available,3.3,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...
3,UPSRTC - RKD0128,Janrath AC Seater 2+2,20:01,01:01,05h 00m,598,36 Seats available,4.3\n12,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...
4,UPSRTC - BRT0007,Ordinary Non AC Seater 2+3,21:25,04:00,06h 35m,448,52 Seats available,3.3,Bareilly to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...
...,...,...,...,...,...,...,...,...,...,...
375,International Tourist Centre,VE A/C Seater / Sleeper (2+1),22:25,05:05,06h 40m,490,38 Seats available,4.2\n511,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...
376,India Tours & Travels (GetBookCab),A/C Seater / Sleeper (2+2),23:00,06:30,07h 30m,391,25 Seats available,4.2\n389,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...
377,Laxmi holidays,VE A/C Seater / Sleeper (2+1),22:30,04:30,06h 00m,522,19 Seats available,4.2\n335,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...
378,Rjroadrunner,A/C Seater / Sleeper (2+2),23:00,05:00,06h 00m,599,34 Seats available,New,Delhi to Haldwani,https://www.redbus.in/bus-tickets/delhi-to-hal...


WEST BENGAL BUS DETAILS

In [13]:
df_wb=pd.read_csv('route_link_data/df_wb.csv')
df_wb

,Route_name,Route_link
0,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...
1,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
2,Barasat (West Bengal) to Digha,https://www.redbus.in/bus-tickets/barasat-west...
3,Suri to Kolkata,https://www.redbus.in/bus-tickets/suri-to-kolkata
4,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...
5,Bolpur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/bolpur-west-...
6,Kolkata to Suri,https://www.redbus.in/bus-tickets/kolkata-to-suri
7,Digha to Kolkata,https://www.redbus.in/bus-tickets/digha-to-kol...
8,Kolkata to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-d...
9,Habra to Digha,https://www.redbus.in/bus-tickets/habra-to-digha


In [67]:
#retrive the bus details
driver_k = webdriver.Chrome()
Bus_names_k = []
Bus_types_k = []
Departure_k = []
Arrival_k = []
Ratings_k = []
Total_Duration_k = []
Prices_k = []
Seats_Available_k = []
Route_names = []
Route_links = []

#df.iterrows() is used for traversing the Route_link and Route_name simultaneously and is traversed by using 'r' 
for i,r in df_wb.iterrows(): 
    link=r["Route_link"]   # in each index each row is accessed so for example --> Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam
    routes=r["Route_name"]  # in above r["Route_link"] in each index and r['Route_name'] gives the values from the df as Pandas series ... for example if in i = 0 and r['Route_name'] we get Kozhikode to ernakulam 

# Loop through each link
    driver_k.get(link)  #in particular bus link we have to extract...so to load that page we use get()
    time.sleep(2)   #time for waiting before interacting and executing next step dynamically

    # Click on elements to reveal bus details
    elements = driver_k.find_elements(By.XPATH, f"//a[contains(@href, '{link}')]")  # in href="https://www.redbus.in/bus-tickets/kozhikode-to-ernakulam" the element is loaded 
    for element in elements: 
        element.click()   #the text element has anchor tag so if we click it loads the page where we need to extract the bus details
        time.sleep(2)   #now we have clicked system needs to interact...so if we pause for 2 seconds ...system dynamically gets ready within that time and we can interact further for execution
        
    # click elements to views bus
    try:
        clicks = driver_k.find_element(By.XPATH, "//div[@class='button']")  #the link opens page loads...the view busses button to be clicked 
        clicks.click()  #automatically simulates a mouse click event on the element gathered from XPATH
    except:
        pass 
    time.sleep(2)
    
    scrolling = True
    while scrolling:
        old_page_source = driver_k.page_source    #page_source gets the DOM structure after any dynamic event like click or action performed like scrolling
        
        # Use ActionChains to perform a PAGE_DOWN
        ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()  #simulation of keyboard occurs from the actionChain class for user interaction in that particular webdriver instance driver_k.. 
                                                                    #send_keys is a method where Keys.PageDown using keyboard keys helps to scroll long pages easily 
                                                                    #perform - executes the simulation
                                                                    
        
        time.sleep(5)  
        
        new_page_source = driver_k.page_source   #DOM Element is stored as string in new_page_source
        
        if new_page_source == old_page_source:
            scrolling = False                    #if new DOM element string is same as old DOM element string

    # Extract bus details
    bus_name = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")   #each row with each index is iterated using the XPATH, 
    bus_type = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")  
    start_time = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
    end_time = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
    total_duration = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
    
    price = driver_k.find_elements(By.XPATH, '//div[@class="fare d-block"]//span')
    seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

    try:
        rating = driver_k.find_elements(By.XPATH,"//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
    except:
        continue
    # Append data to respective lists
    for bus in bus_name:
        Bus_names_k.append(bus.text)
        Route_links.append(link)
        Route_names.append(routes)
    for bus_type_elem in bus_type:
        Bus_types_k.append(bus_type_elem.text)
    for start_time_elem in start_time:
        Departure_k.append(start_time_elem.text)
    for end_time_elem in end_time:
        Arrival_k.append(end_time_elem.text)
    for total_duration_elem in total_duration:
        Total_Duration_k.append(total_duration_elem.text)
    for ratings in rating:
        Ratings_k.append(ratings.text)
    for price_elem in price:
        Prices_k.append(price_elem.text)
    for seats_elem in seats:
        Seats_Available_k.append(seats_elem.text)
        
print("Successfully Completed")

Successfully Completed


In [68]:
# from list to convert data frame
data = {
    'Bus_name': Bus_names_k,
    'Bus_type': Bus_types_k,
    'Start_time': Departure_k,
    'End_time': Arrival_k,
    'Total_duration': Total_Duration_k,
    'Price': Prices_k,
    "Seats_Available":Seats_Available_k,
    "Ratings":Ratings_k,
    'Route_name': Route_names,
    'Route_link': Route_links
}

df_buses_11 = pd.DataFrame(data)
#convert dataframe to csv
df_buses_11.to_csv("df_bus11.csv",index=False)

In [69]:
df_buses_11

,Bus_name,Bus_type,Start_time,End_time,Total_duration,Price,Seats_Available,Ratings,Route_name,Route_link
0,SBSTC-BANKURA - KARUNAMOYEE - A/C - 15:15 - 1500,AC Seater (2+2),17:00,21:00,04h 00m,300,33 Seats available,2.0\n94,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...
1,SBSTC-ASANSOL - KARUNAMOYEE - 16:10 (BEL DEPOT...,Non AC Seater (2+3),17:25,21:25,04h 00m,155,50 Seats available,3.5\n74,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...
2,SBSTC-ASANSOL - KOLKATA - 16:30 (HOWRAH DEPOT)...,Non AC Seater (2+3),17:40,21:10,03h 30m,150,50 Seats available,4.1\n38,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...
3,SBSTC-ASANSOL - KARUNAMOYEE - 16:40 (ASANSOL D...,Non AC Seater (2+3),17:50,22:00,04h 10m,155,48 Seats available,3.8\n74,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...
4,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo B11R Multi Axle Seater (2+2),16:00,20:00,04h 00m,486,36 Seats available,4.3\n367,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...
...,...,...,...,...,...,...,...,...,...,...
92,SHYAMOLI Pvt. Ltd.(Karunamoyee),Volvo A/C Seater (2+2),17:30,22:30,05h 00m,524,33 Seats available,4.1\n134,Kolkata to Asansol (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-a...
93,Greenline,Volvo A/C Seater (2+2),18:05,23:05,05h 00m,524,37 Seats available,4.0\n32,Kolkata to Asansol (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-a...
94,Shyamoli Paribahan Pvt Ltd,Volvo A/C Seater (2+2),18:40,23:25,04h 45m,524,31 Seats available,4.2\n63,Kolkata to Asansol (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-a...
95,Royal Cruiser,Scania Multi-Axle AC Semi Sleeper (2+2),19:30,00:15,04h 45m,509,42 Seats available,3.6\n60,Kolkata to Asansol (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-a...
